In [2]:
from scipy.stats import norm
from math import *

class EquityOption:
    def __init__(self, F='c',S=40, K=35, T=1, V=0.5, R=0.04, D=0.06):
        self.F = F
        self.S = S
        self.K = K 
        self.T = T
        self.V = V
        self.R = R
        self.D = D
        
#     def __str__(self, F, S, K, T, V, R, D):
#         return ('CallFlag = %s \n Spot = %s \n Strike = %s \n Maturity = %s \n Volatility = %s \n RiskFreeRate = %s \n Dividend Yield = %s' %(F, S, K, T, V, R, D))
    
#     def __imul__(self,stock_split):
#         self.S = self.S/stock_split
#         self.K = self.K/stock_split
        
    def BlackScholes(self, F, S, K, T, V, R, D):
        d1 = (log(float(self.S)/self.K)+((self.R-self.D)+self.V*self.V/2.)*self.T)/(self.V*sqrt(self.T))
        d2 = d1-self.V*sqrt(self.T)
        if F == 'c':
            optionprice = self.S*exp(-self.D*self.T)*norm.cdf(d1)-self.S*exp(-self.R*self.T)*norm.cdf(d2)
        else:
            optionprice = self.K*exp(-self.R*self.T)*norm.cdf(-d2)-self.S*exp(-self.D*self.T)*norm.cdf(-d1)
        return optionprice
    
    def Greeks(self, F, S, K, T, V, R, D):
        d1 = (log(float(self.S)/self.K)+((self.R-self.D)+self.V*self.V/2.)*self.T)/(self.V*sqrt(self.T))
        d2 = d1-self.V*sqrt(self.T)
        T_sqrt = sqrt(self.T)
        if F == 'c':
            Delta = norm.cdf(d1)
            Gamma = norm.pdf(d1)/(self.S*self.V*T_sqrt)
            Theta = - (self.S*self.V*norm.pdf(d1))/(2*T_sqrt) - self.R*self.K*exp(-self.R*self.T)*norm.cdf(d2)
            Vega = self.S * T_sqrt*norm.pdf(d1)
        else:
            Delta = -norm.cdf(-d1)
            Gamma = norm.pdf(d1)/(self.S*self.V*T_sqrt)
            Theta = -(self.S*self.V*norm.pdf(d1)) / (2*T_sqrt)+ self.R*self.K * exp(-self.R*self.T) * norm.cdf(-d2)
            Vega = self.S * T_sqrt * norm.pdf(d1)    
        return Delta, Gamma, Theta, Vega     
    
    #input random value to test (direct number not calculation from BS)
    @staticmethod
    def biset_iv(value, prec, F, S, K, T, R, D):
        begin=0
        end=5
        V=(begin+end)/2
        E=EquityOption(F, S, K, T, V, R, D)
        d1,d2 = E.__d1_d2__(S, K, T, V, R, D)
        value_new = E.BlackScholes(d1, d2, F, S, K, T, V, R, D) 
        for i in range(100):
            if value_new - value>0:
                V = (begin+V)/2
                d1,d2 = E.__d1_d2__(S, K, T, V, R, D)
                value_new = E.BlackScholes(d1, d2, F, S, K, T, V, R, D)
            else:
                V =(V+end)/2
                d1,d2 = E.__d1_d2__(S, K, T, V, R, D)
                value_new = E.BlackScholes(d1,d2, F, S, K, T, V, R, D)
            if (abs(value_new - value)<prec):
                return V
        return V
    
#     @staticmethod
#     def __d1_d2__(S, K, T, V, R, D):
#         d1 = (log(float(S)/K)+((R-D)+V*V/2.)*T)/(V*sqrt(T))
#         d2 = d1-V*sqrt(T)
#         return d1,d2
    
 
    #input random value to test (direct number not calculation from BS)
    @staticmethod
    def newton_iv(value, F, S, K, T, R, D):
        T_sqrt = sqrt(T)
        prec = 0.0001
        V=0.5
        for i in range(100):
            E=EquityOption(F, S, K, T, V, R, D)
            d1,d2 = E.__d1_d2__(S, K, T, V, R, D)
            price= E.BlackScholes(d1,d2, F, S, K, T, V, R, D)
            if F == 'c':
                Vega = S * T_sqrt*norm.pdf(d1)
            else:
                Vega = S * T_sqrt * norm.pdf(d1)
            diff=value-price
            if (abs(diff)<prec):
                return V
            V=V+diff/Vega 
        return V
    


kwargs = dict(F='c',S=40, K=35, T=1, V=0.5, R=0.04, D=0.06)  
E1= EquityOption(**kwargs)
kwargs1 = dict(S=40, K=35, T=1, V=0.5, R=0.04, D=0.06) 
d1,d2 = E1.__d1_d2__(**kwargs1)
Delta, Gamma, Theta, Vega = E1.Greeks(d1,d2,**kwargs)
print(E1.__str__(**kwargs))
price = E1.BlackScholes(d1,d2,**kwargs)
print("Option Price = ", price)
kwargs2 = dict(F='c',S=40, K=35, T=1, R=0.04, D=0.06) 
prec = 0.00001
IV_BISET = E1.biset_iv(6.87, prec, **kwargs2)
print("Implied Volatility Biset = ", IV_BISET)
IV_NEWTON = E1.newton_iv(6.87, **kwargs2)
print("Implied Volatility NR = ", IV_NEWTON)




CallFlag = c 
 Spot = 40 
 Strike = 35 
 Maturity = 1 
 Volatility = 0.5 
 RiskFreeRate = 0.04 
 Dividend Yield = 0.06
Option Price =  6.877717095221037
Implied Volatility Biset =  2.6666666666666665
Implied Volatility NR =  0.499496160052575


In [5]:
kwargs = dict(F='c',S=100, K=100, T=0.5, V=0.5, R=0, D=0)  
E1= EquityOption(**kwargs)
kwargs1 = dict(S=100, K=100, T=0.5, V=0.5, R=0, D=0) 
d1,d2 = E1.__d1_d2__(**kwargs1)
price = E1.BlackScholes(d1,d2,**kwargs)
print("Option Price = ", price)

Option Price =  14.031620480133384


In [6]:
kwargs = dict(F='p',S=100, K=100, T=0.5, V=0.5, R=0, D=0)  
E1= EquityOption(**kwargs)
kwargs1 = dict(S=100, K=100, T=0.5, V=0.5, R=0, D=0) 
d1,d2 = E1.__d1_d2__(**kwargs1)
price = E1.BlackScholes(d1,d2,**kwargs)
print("Option Price = ", price)

Option Price =  14.031620480133384
